In [ ]:
!pip install image-classifiers==1.0.0b1
!pip install keras_applications --no-deps
!pip install tensorflow==2.10

In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds
import pandas as pd
import numpy as np
import os
from PIL import Image
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import Callback
from keras.callbacks import ModelCheckpoint
print(tf.__version__)

2.10.0


In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


### Load Adience dataset

In [4]:
fold0 = tf.data.Dataset.load('/content/drive/MyDrive/data/saved_data/cv_fold0')
fold1 = tf.data.Dataset.load('/content/drive/MyDrive/data/saved_data/cv_fold1')
fold2 = tf.data.Dataset.load('/content/drive/MyDrive/data/saved_data/cv_fold2')
fold3 = tf.data.Dataset.load('/content/drive/MyDrive/data/saved_data/cv_fold3')
fold4 = tf.data.Dataset.load('/content/drive/MyDrive/data/saved_data/cv_fold4')
train = fold0.concatenate(fold1)
train = train.concatenate(fold2)
val = fold3
test = fold4

### Load base model pretrained on CELEBA

In [5]:
base_model = tf.keras.models.load_model('./drive/MyDrive/data/saved_model/ResNet_celeba.h5')

- We want to add layers to CELEBA model, so we do not use the model output, we use the layer before all output layers (named 'relu1')
- We create a `new_base_model` with same input but use 'relu1' as output, then add layers we want to this `new_base_model` below

In [6]:
base_model.layers[-42] # this is where 'relu1' is since last 40 are output layers and last 41 is pooling

In [7]:
# new_base_model with 'relu1' as output
new_base_model = tf.keras.Model(inputs = base_model.input,
                                outputs = [base_model.get_layer('relu1').output])

# iterate over all the layers in orignal base_model to load weights for new_base_model
weights = [layer.get_weights() for layer in base_model.layers[:-42]]
for layer, weight in zip(new_base_model.layers, weights):
  layer.set_weights(weight)

- We want to freeze all layers except the last 10 layers. <br> Hence, we make all layers except the last 10 layers non-trainable

In [8]:
for layer in new_base_model.layers[:-10]:
  layer.trainable = False

### Create model for Adience based on pretrained model

In [9]:
def create_model(base_model):
  ''' 
  Add squeeze-excite block on top of last layer before classificatiion layer

  Args:
        base_model: ResNet34 model pretrained on CELEBA
        
  Returns:
        new model used to train age_gender
  '''


  img = tf.keras.Input(shape=(224,224,3))

  # add data augmentation
  img = tf.keras.layers.RandomFlip(mode='horizontal_and_vertical')(img)
  img = tf.keras.layers.RandomRotation(0.1)(img)
  img = tf.keras.layers.RandomContrast(0.1)(img)

  # pass the image through base model trained on CELEBA
  pretrained_output = base_model(img)

  # squeeze
  main_branch = tf.keras.layers.GlobalAveragePooling2D()(pretrained_output)
  # excite
  main_branch = tf.keras.layers.Dense(256, activation = 'relu')(main_branch)
  main_branch = tf.keras.layers.Dense(512, activation='sigmoid')(main_branch)
  # scale back to same dim
  main_branch = tf.keras.layers.Multiply()([pretrained_output, main_branch])
  main_branch = tf.keras.layers.GlobalAveragePooling2D()(main_branch)
  # Dropout layer to prevent overfitting
  main_branch = tf.keras.layers.Dropout(0.6)(main_branch)

  # add Dense before age and gender output, also add Dropout and regularization to prevent overfitting
  gender_branch = tf.keras.layers.Dense(128, activation='relu')(main_branch)
  gender_branch = tf.keras.layers.Dropout(0.8)(gender_branch)
  gender_branch = tf.keras.layers.Dense(1, activation='sigmoid', kernel_regularizer=l2(0.1), name='gender_output')(gender_branch)
  
  age_branch = tf.keras.layers.Dense(128, activation='relu')(main_branch)
  age_branch = tf.keras.layers.Dropout(0.8)(age_branch)
  age_branch = tf.keras.layers.Dense(8, activation='softmax', kernel_regularizer=l2(0.1), name='age_output')(age_branch)

  model = tf.keras.Model(inputs = img,
                         outputs = [gender_branch, age_branch])

  return model

In [10]:
# create model and compile it
model = create_model(new_base_model)
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.0003),
    loss={'gender_output': 'binary_crossentropy', 'age_output': 'categorical_crossentropy'},
    metrics={'gender_output': 'accuracy', 'age_output': 'accuracy'},
)

# model summary
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 model (Functional)             (None, 7, 7, 512)    21302473    ['input_2[0][0]']                
                                                                                                  
 global_average_pooling2d (Glob  (None, 512)         0           ['model[1][0]']                  
 alAveragePooling2D)                                                                              
                                                                                            

In [ ]:
# create new metrics: mean of age + gen accuracy for train and val
class MeanAccCallback(Callback):
  def on_epoch_end(self, epoch, logs={}):
    logs['mean_age_gen_acc'] = (logs.get('gender_output_accuracy') + logs.get('age_output_accuracy'))/2
    logs['mean_val_age_gen_acc'] = (logs.get('val_gender_output_accuracy') + logs.get('val_age_output_accuracy'))/2

In [ ]:
# save the model with best mean val_age_gen_acc
file_path = "/content/drive/MyDrive/data/saved_model/celeba_age_gen_plus.h5"

checkpoint = ModelCheckpoint(file_path, monitor='mean_val_age_gen_acc', verbose=1, save_best_only=True, mode='max')

In [ ]:
hist = model.fit(train.shuffle(1024).batch(64),
                 epochs=20, 
                 validation_data = val.shuffle(1024).batch(64),
                 callbacks = [
                     EarlyStopping(monitor = 'val_loss', patience=5),
                     MeanAccCallback(),
                     checkpoint, ])

Epoch 1/20
118/118 [==============================] - ETA: 0s - loss: 4.4290 - gender_output_loss: 0.7620 - age_output_loss: 2.1941 - gender_output_accuracy: 0.5248 - age_output_accuracy: 0.1859
Epoch 1: mean_val_age_gen_acc improved from -inf to 0.42355, saving model to /content/drive/MyDrive/data/saved_model/celeba_age_gen_plus.h5
118/118 [==============================] - 27s 190ms/step - loss: 4.4290 - gender_output_loss: 0.7620 - age_output_loss: 2.1941 - gender_output_accuracy: 0.5248 - age_output_accuracy: 0.1859 - val_loss: 3.9434 - val_gender_output_loss: 0.6849 - val_age_output_loss: 2.0306 - val_gender_output_accuracy: 0.5341 - val_age_output_accuracy: 0.3130 - mean_age_gen_acc: 0.3554 - mean_val_age_gen_acc: 0.4235
Epoch 2/20
118/118 [==============================] - ETA: 0s - loss: 3.7230 - gender_output_loss: 0.6875 - age_output_loss: 2.0087 - gender_output_accuracy: 0.5559 - age_output_accuracy: 0.2337
Epoch 2: mean_val_age_gen_acc improved from 0.42355 to 0.43583, savi

### Graphs for loss and accuracy against epochs

### Save and load trained model

In [11]:
model = tf.keras.models.load_model('./drive/MyDrive/data/saved_model/celeba_age_gen_plus.h5')

In [12]:
model.summary()

Model: "model_21"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_22 (InputLayer)          [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 model_20 (Functional)          (None, 7, 7, 512)    21302473    ['input_22[0][0]']               
                                                                                                  
 global_average_pooling2d_20 (G  (None, 512)         0           ['model_20[0][0]']               
 lobalAveragePooling2D)                                                                           
                                                                                           

### Evaluate trained model on test data

In [13]:
metrics = model.evaluate(test.batch(64), return_dict = True)

36/36 [==============================] - 23s 386ms/step - loss: 2.6899 - gender_output_loss: 0.9466 - age_output_loss: 1.6734 - gender_output_accuracy: 0.7215 - age_output_accuracy: 0.3516


In [14]:
metrics

{'loss': 2.6898880004882812,
 'gender_output_loss': 0.9465535283088684,
 'age_output_loss': 1.6733922958374023,
 'gender_output_accuracy': 0.7214794754981995,
 'age_output_accuracy': 0.35160428285598755}

In [15]:
test_ds = test.batch(1)
preds = model.predict(test_ds)

2244/2244 [==============================] - 22s 9ms/step


In [16]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
def gender_metrics(preds, ds):
  gender_true = []
  for i in ds.as_numpy_iterator():
    # take label dict
    label_dict = i[1] 
    # append all values with key 'gender_output' to a list
    gender = label_dict['gender_output'][0] 
    # append all gender to a list
    gender_true.append(gender) 
  gender_preds = np.where(preds[0].ravel()>0.5, 1, 0)
  return {'accuracy': accuracy_score(gender_true, gender_preds), 
          'f1': f1_score(gender_true, gender_preds), 
          'precision': precision_score(gender_true, gender_preds), 
          'recall': recall_score(gender_true, gender_preds)}

In [17]:
gender_metrics(preds, test_ds)

{'accuracy': 0.7214795008912656,
 'f1': 0.7526711515631183,
 'precision': 0.7150375939849624,
 'recall': 0.7944862155388471}